In [26]:
import wandb

In [27]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /opt/ml/.netrc


True

In [2]:
import torch
torch.__version__

'1.7.1+cu101'

In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

ImportError: /opt/conda/lib/python3.7/site-packages/mmcv/_ext.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN3c104impl23ExcludeDispatchKeyGuardC1ENS_11DispatchKeyE

In [7]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=90
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco'

cfg.model.bbox_head.num_classes = 11
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.log_config.hooks = [dict(type='TextLoggerHook'), dict(type='WandbLoggerHook')]
cfg.checkpoint_config['max_keep_ckpts']=5
cfg.evaluation.save_best = 'bbox_mAP_50'

cfg.lr_config.step = [60, 90]
cfg.runner.max_epochs = 100

In [8]:
print(cfg.lr_config)

{'policy': 'step', 'warmup': 'linear', 'warmup_iters': 500, 'warmup_ratio': 0.1, 'step': [60, 90]}


In [9]:
print(cfg.runner)

{'type': 'EpochBasedRunner', 'max_epochs': 100}


In [10]:
model = build_detector(cfg.model)

2021-05-14 09:19:56,944 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-14 09:19:56,945 - mmdet - INFO - Use load_from_torchvision loader
2021-05-14 09:19:57,191 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offset.bias, layer3.4.conv2.conv_offset.weight, layer3.4.conv2.conv_offset.bias, layer3.5.conv2.conv_offset.weight, layer3.5

In [11]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.66s)
creating index...
index created!


In [12]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-14 09:20:16,386 - mmdet - INFO - Start running, host: root@bc720df3e041, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco
2021-05-14 09:20:16,389 - mmdet - INFO - workflow: [('train', 1)], max: 100 epochs


Done (t=0.87s)
creating index...
index created!


/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
wandb: Currently logged in as: k1smet14 (use `wandb login --relogin` to force relogin)


2021-05-14 09:20:49,774 - mmdet - INFO - Epoch [1][50/655]	lr: 1.882e-03, eta: 10:31:38, time: 0.579, data_time: 0.050, memory: 1887, loss_cls: 0.7547, loss_bbox: 0.8351, loss_bbox_rf: 1.1405, loss: 2.7302, grad_norm: 12.0811
2021-05-14 09:21:16,136 - mmdet - INFO - Epoch [1][100/655]	lr: 2.782e-03, eta: 10:02:55, time: 0.527, data_time: 0.007, memory: 1887, loss_cls: 0.7641, loss_bbox: 0.7650, loss_bbox_rf: 1.0197, loss: 2.5487, grad_norm: 7.5138
2021-05-14 09:21:42,538 - mmdet - INFO - Epoch [1][150/655]	lr: 3.682e-03, eta: 9:53:20, time: 0.528, data_time: 0.007, memory: 1887, loss_cls: 0.8474, loss_bbox: 0.7158, loss_bbox_rf: 0.9570, loss: 2.5203, grad_norm: 6.9503
2021-05-14 09:22:09,566 - mmdet - INFO - Epoch [1][200/655]	lr: 4.582e-03, eta: 9:51:44, time: 0.541, data_time: 0.006, memory: 1887, loss_cls: 0.9136, loss_bbox: 0.6838, loss_bbox_rf: 0.9095, loss: 2.5069, grad_norm: 5.7465
2021-05-14 09:22:35,970 - mmdet - INFO - Epoch [1][250/655]	lr: 5.482e-03, eta: 9:47:53, time: 0.5

...

Loading and preparing results...
DONE (t=0.11s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.61s).
Accumulating evaluation results...


2021-05-14 20:27:54,981 - mmdet - INFO - Epoch(val) [100][655]	bbox_mAP: 0.1900, bbox_mAP_50: 0.2770, bbox_mAP_75: 0.1960, bbox_mAP_s: 0.0250, bbox_mAP_m: 0.1530, bbox_mAP_l: 0.4240, bbox_mAP_copypaste: 0.190 0.277 0.196 0.025 0.153 0.424


DONE (t=0.84s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.190
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.277
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.196
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.025
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.153
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.424
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.315
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.315
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.315
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.078
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.267
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.589


train/loss_cls,0.19205
train/loss_bbox,0.07529
train/loss_bbox_rf,0.09612
train/loss,0.36346
train/grad_norm,1.19327
learning_rate,0.0001
momentum,0.9
_runtime,40058
_timestamp,1621024074
_step,65501
val/bbox_mAP,0.19


train/loss_cls,█▇█▇▇▇▇▆▆▆▆▅▅▄▅▄▄▄▃▃▃▃▃▅▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,█▆▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox_rf,█▆▅▅▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▃▂▂▂▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▆▆▆▅▅▅▅▄▅▄▄▄▄▄▃▄▃▃▃▃▂▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▃▃▄▃▄▄▅▅▅▅▅▅▅▅▅▅▅▆▄▄▄▃▃█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
learning_rate,████████████████████████▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/bbox_mAP,▁▂▄▄▅▆▆▆▆▆▆▇▇▇▇▆▇▇▇▇▆██▆████████████████


In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [6]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/vfnet_x101_64x4d_fpn_mstrain_2x_coco.py')

PREFIX = '../../input/data/'

epoch = 24

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=90
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet_x101_64x4d_fpn_mstrain_2x_coco'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'best_bbox_mAP_50.pth')

In [7]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [8]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-13 06:13:54,379 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-13 06:13:54,381 - mmdet - INFO - Use load_from_openmmlab loader


Use load_from_local loader


OSError: ./work_dirs/vfnet_x101_64x4d_fpn_mstrain_2x_coco/best_bbox_mAP_50.pth is not a checkpoint file

In [16]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 8.5 task/s, elapsed: 98s, ETA:     0s

In [17]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.06772634 228.28596 149.0423 249.09592 236....,batch_01_vt/0021.jpg
1,0 0.1134868 284.8081 151.96843 486.8694 434.62...,batch_01_vt/0028.jpg
2,0 0.11189955 68.59096 171.01566 372.83655 379....,batch_01_vt/0031.jpg
3,0 0.057285193 29.397675 124.40297 385.30334 37...,batch_01_vt/0032.jpg
4,0 0.17107947 237.8103 266.21378 383.67017 377....,batch_01_vt/0070.jpg
